In [ ]:
#Cmoney股市爆料同學會留言爬蟲
import requests
from bs4 import BeautifulSoup
import json
import datetime
import os
import sys
import requests
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

#建立時間的條件，用來篩選出今天盤後到明天盤前的資料

yesterday2pm = (datetime.datetime.now() + datetime.timedelta(-1)).replace(hour=14, minute=0, second=0, microsecond=0)
today9am = datetime.datetime.now().replace(hour=9, minute=0, second=0, microsecond=0)
tomorrow12am = (datetime.datetime.now() + datetime.timedelta(+1)).replace(hour=0, minute=0, second=0, microsecond=0)
today2pm = datetime.datetime.now().replace(hour=14, minute=0, second=0, microsecond=0)
today12am = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

#要設定成自己的路徑
driver = webdriver.Chrome('/Users/User/Desktop/Python/chromedriver.exe') 
driver.get("https://www.cmoney.tw/follow/channel/stock-TWA00?chart=d&type=Personal") #開啟模擬瀏覽器
js = "var q=document.documentElement.scrollTop=10000"  #selenium 滾輪拉到底設定


url="https://www.cmoney.tw/follow/channel/getdata/articlelistofstockv2?articleCategory=Personal&channelId=103681&size=21&sTime=&articleSortType=latest&articleSortCount=0&isIncludeLimitedAskArticle=false&_=1606584302518"
re=requests.post(url)
data=json.loads(re.text)

#這段主要是為了selenium模擬下拉以後網址會改變，下面這三個是他的起始值和規律
skipCount=21                                #起始沒有，下拉一次從21開始，第一次以後每次+10
size=10                                     #起使=21 接下來每次都固定10
articleId=data[20]["ArtId"]                 # articleId = 每一個Json網頁的最後一人發表的ID

nowtime = datetime.datetime.now() #現在時間
if 0 < nowtime.weekday() < 4 :
    if today2pm < nowtime < tomorrow12am:       #假設現在時間介於 今天下午兩點與今天半夜12點  代表是今天使用
        
        #在還沒用selenium往下拉之前只能爬20筆留言的迴圈
        n=0                         #n是計數器功能，用來顯示每一則留言的序號        
        while n<=20:               #第一篇(1~20)爬蟲
            data_content=data[n]["ArtCtn"]
            data_time=data[n]["ArtCteTm"]
            data_content_final = str(n)+".  "+data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
            date_time = data_time.replace("/","-")
            date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M') #算出爬出資料的時間，若超過設定的範圍則停止
            n+=1
            if date_time_final > today2pm:
                print(data_content_final)
                print(date_time_final)
                
                
        #下面的while是用selenium往下拉之後爬取留言的迴圈  (經觀察，往下拉一次只能爬取9筆留言)
        while date_time_final > today2pm:
            for i in range(0,10):
                n+=1                    #n是計數器功能，用來顯示每一則留言的序號
                
                driver.execute_script(js) #啟動模擬滑鼠滾輪下拉網頁
                
                #下拉以後網址會改變，找到規律以後利用規律更改網址來配合下拉
                url = 'https://www.cmoney.tw/follow/channel/getdata/articlelistmoreofstockv2?articleCategory=Personal&channelId=103681&articleId='+str(articleId)+"&size="+str(size)+"&skipCount="+str(skipCount)+"&articleSortCount=0&sTime=&articleSortType=latest&isIncludeLimitedAskArticle=false&_=1606990418284"
                articleId=data[9]["ArtId"]
                skipCount+=10
                size=10
                re=requests.post(url)
                data=json.loads(re.text)
                data_content=data[i]["ArtCtn"]
                data_time=data[i]["ArtCteTm"]
                data_content_final = str(n)+".  "+data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                date_time = data_time.replace("/","-")
                date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M') #算出爬出資料的時間，若超過設定的範圍則停止
                if date_time_final > today2pm:  #判斷爬出的貼完時間是否符合條件
                    print(data_content_final)
                    print(date_time_final)
        else:
            driver.close()    #不符合條件就關閉chrome模擬器停止爬蟲
